1. https://leetcode.com/problems/paint-house/

There is a row of n houses, where each house can be painted one of three colors: red, blue, or green. The cost of painting each house with a certain color is different. You have to paint all the houses such that no two adjacent houses have the same color.

The cost of painting each house with a certain color is represented by an n x 3 cost matrix costs.

For example, costs[0][0] is the cost of painting house 0 with the color red; costs[1][2] is the cost of painting house 1 with color green, and so on...
Return the minimum cost to paint all houses.

 

Example 1:

Input: costs = [[17,2,17],[16,16,5],[14,3,19]]
Output: 10
Explanation: Paint house 0 into blue, paint house 1 into green, paint house 2 into blue.
Minimum cost: 2 + 5 + 3 = 10.
Example 2:

Input: costs = [[7,6,2]]
Output: 2
 

Constraints:

costs.length == n
costs[i].length == 3
1 <= n <= 100
1 <= costs[i][j] <= 20

## My Notes
The bruteforce approach is to recursively try all the permutations and eliminate such permutation that does have two adjacent houses are painted in same color. A better approach is to try permutations that have adjacent houses painted in different color & pick the minimum cost from it. The recursive/memoized solution is easy to understand than the bottom-up approach. The bottom-up approach uses the same costs array to store the minimum cost instead of auxilary space.

Time complexity: O(n)

Space complexity: O(1) - Bottom-up solution, O(n) memoized recursive solution

In [1]:
from typing import List
from functools import lru_cache


class Solution:
    def minCost(self, costs: List[List[int]]) -> int:
        if len(costs) == 0: return 0
        for n in range(len(costs) - 2, -1, -1):
            costs[n][0] += min(costs[n + 1][1], costs[n + 1][2])
            costs[n][1] += min(costs[n + 1][0], costs[n + 1][2])
            costs[n][2] += min(costs[n + 1][0], costs[n + 1][1])
        return min(costs[0])

    def minCostMemoized(self, costs: List[List[int]]) -> int:
        @lru_cache(maxsize = None)
        def dp(n, color):
            total_cost = costs[n][color]

            if n == len(costs) - 1:
                return total_cost
            elif color == 0:
                total_cost += min(dp(n + 1, 1), dp(n + 1, 2))
            elif color == 1:
                total_cost += min(dp(n + 1, 0), dp(n + 1, 2))
            elif color == 2:
                total_cost += min(dp(n + 1, 0), dp(n + 1, 1))
            return total_cost
        return min(dp(0, 0), dp(0, 1), dp(0, 2))


def main():
    s = Solution()

    print(s.minCost([[17,2,17],[16,16,5],[14,3,19]]))
    print(s.minCost([[7,6,2]]))
    print(s.minCost([[3,5,3],[6,17,6],[7,13,18],[9,10,18]]))


if __name__ == "__main__":
    main()


10
2
26


2. https://leetcode.com/problems/paint-house-ii/

There are a row of n houses, each house can be painted with one of the k colors. The cost of painting each house with a certain color is different. You have to paint all the houses such that no two adjacent houses have the same color.

The cost of painting each house with a certain color is represented by an n x k cost matrix costs.

For example, costs[0][0] is the cost of painting house 0 with color 0; costs[1][2] is the cost of painting house 1 with color 2, and so on...
Return the minimum cost to paint all houses.

 

Example 1:

Input: costs = [[1,5,3],[2,9,4]]
Output: 5
Explanation:
Paint house 0 into color 0, paint house 1 into color 2. Minimum cost: 1 + 4 = 5; 
Or paint house 0 into color 2, paint house 1 into color 0. Minimum cost: 3 + 2 = 5.
Example 2:

Input: costs = [[1,3],[2,4]]
Output: 5
 

Constraints:

costs.length == n
costs[i].length == k
1 <= n <= 100
2 <= k <= 20
1 <= costs[i][j] <= 20

## My Notes

This problem is like paint house I problem with a generalization that there are "k" colors instead of 3 colors. The time optimized solution is to track first and second minimum costs from previous row. Then update the costs for current row.

In [2]:
from typing import List
from functools import lru_cache


class Solution:
    def minCostII(self, costs: List[List[int]]) -> int:
        n = len(costs)
        if n == 0: return 0
        k = len(costs[0])

        for house in range(1, n):
            # Find the colors with the minimum and second to minimum
            # in the previous row.
            min_color = second_min_color = None
            for color in range(k):
                cost = costs[house - 1][color]
                if min_color is None or cost < costs[house - 1][min_color]:
                    second_min_color = min_color
                    min_color = color
                elif second_min_color is None or cost < costs[house - 1][second_min_color]:
                    second_min_color = color
            # And now update the costs for the current row.
            for color in range(k):
                if color == min_color:
                    # can't use min_color as no two adjacent houses can have same color
                    costs[house][color] += costs[house - 1][second_min_color]
                else:
                    costs[house][color] += costs[house - 1][min_color]

        #The answer will now be the minimum of the last row.
        return min(costs[-1])
        
    def minCostIISpace1(self, costs: List[List[int]]) -> int:
        n = len(costs)
        k = len(costs[0])
        if n == 0:
            return 0
        for house in range(1, n):
            for color in range(k):
                best = float("inf")
                for prev_color in range(k):
                    if color == prev_color:
                        continue
                    best = min(best, costs[house - 1][prev_color])
                costs[house][color] += best
        return min(costs[-1])


    def minCostIIMemoized(self, costs: List[List[int]]) -> int:
        n = len(costs)
        k = len(costs[0])

        @lru_cache(maxsize = None) 
        def dp(house, color):
            if n - 1 == house:
                return costs[house][color]
            cost = float("inf")

            for next_color in range(k):
                if color == next_color:
                    continue
                cost = min(cost, dp(house + 1, next_color))
            return cost + costs[house][color]

        cost = float("inf")
        for color in range(k):
            cost = min(cost, dp(0, color))

        return cost


def main():
    s = Solution()

    print(s.minCostII([[1,5,3],[2,9,4]]))
    print(s.minCostII( [[1,3],[2,4]]))


if __name__ == "__main__":
    main()


5
5
